In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import math

In [18]:
data = pd.read_csv('ung thư.csv')
cate_data = pd.read_csv('adult-Copy1.csv')
cate_data.columns = cate_data.columns.str.strip()

In [19]:
cate_data = cate_data.drop(columns = ['2174','0'])


In [20]:
cate_data.nunique()

39                  73
State-gov            9
77516            21647
Bachelors           16
13                  16
Never-married        7
Adm-clerical        15
Not-in-family        6
White                5
Male                 2
40                  94
United-States       42
<=50K                2
dtype: int64

# Split data

In [21]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X1_train,x1_test,y1_train,y1_test = train_test_split(cate_data.iloc[:,:-1],cate_data.iloc[:,-1],test_size=0.25,random_state=42)

In [23]:
data_train = pd.merge(X_train,y_train,left_index=True,right_index=True)
data1_train = pd.merge(X1_train,y1_train,left_index=True,right_index=True)

# Naive bayes classifier

In [24]:
def split_x_train(data_train,target_column):
    grouped = data_train.groupby(target_column)
    subframes = {}
    for category, group in grouped:
        subframes[category] = group.iloc[:,:-1]
        
    return subframes

In [25]:
def find_llk_continuous(data):
    mean = data.mean(axis = 0)
    std = data.std(axis = 0)
    return [mean,std]

def find_llk_categorical(data):
    return data.value_counts(normalize = True)

In [26]:
class Naive_bayes_classifier:
    
    def __init__(self,data,y):
        self._X = split_x_train(data,y.name)
        self._y = y
        self_data = data
        self._class_prob = None
        
    def cal_prior(self):
        return y.value_counts(normalize = True)
        
    def cal_likelihood(self,group):
        cate_data = self._X[group].loc[:,self._X[group].nunique() < 5]
        cont_data = self._X[group].loc[:,self._X[group].nunique() > 5]
        llk_cate = None
        llk_cont = None
        if not cate_data.empty:
            llk_cate = find_llk_categorical(cate_data)
        if not cont_data.empty:
            llk_cont = find_llk_continuous(cont_data)
        return [llk_cate,llk_cont]
        
    def cal_posterior(self,group,x_test):
        prior = self.cal_prior()
        llk = self.cal_likelihood(group)
        cate_data = x_test.loc[:,x_test.nunique() < 5]
        cont_data = x_test.loc[:,x_test.nunique() > 5]
        if not cate_data.empty:
            pass
        if not cont_data.empty:
            pdf = norm.pdf(cont_data, loc=llk[1][0], scale= llk[1][1])        
            prod = pdf.prod(axis = 1)
        return pd.DataFrame(prod * prior[group],columns=[f'{group}'])
    
    def find_argmax(self,x_test):
        df = self.cal_posterior(self._y.unique()[0],x_test)
        for i in self._y.unique()[1:]:
            df =  pd.merge(df,self.cal_posterior(i,x_test),left_index=True,right_index=True)
        # choose appropiate column
        df['highest_prob'] = df.idxmax(axis = 1)  
        return df[['highest_prob']].astype(int)
        

In [27]:
evaluation = Naive_bayes_classifier(data_train,y_train)
y_train.unique()

array([1, 0], dtype=int64)

In [28]:
y_predict = evaluation.find_argmax(x_test=X_test)
y_predict

,highest_prob
0,1
1,0
2,0
3,1
4,1
...,...
138,1
139,1
140,1
141,0


In [29]:
y = pd.DataFrame(y_test).reset_index().drop(columns=['index'])
# accuracy
y_predict['highest_prob']

0      1
1      0
2      0
3      1
4      1
      ..
138    1
139    1
140    1
141    0
142    1
Name: highest_prob, Length: 143, dtype: int32

In [30]:
num_matches = (y_predict['highest_prob'] == y['diagnosis']).sum()
total_obs = y.shape[0]
accuracy = round(num_matches/total_obs,2)
accuracy

0.95

In [34]:
data2 = Naive_bayes_classifier(data1_train,y1_train)
y1_train.unique()

array([' <=50K', ' >50K'], dtype=object)

In [35]:
data2.cal_likelihood('>50K')

KeyError: '>50K'